In [2]:
import requests
import urllib3
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import os.path
%matplotlib inline

In [3]:
company_listing = "http://www.nepalstock.com.np/company?_limit=500"

In [3]:
http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',company_listing)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
company=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
col = 0
notfirstrun = False
for row in rows:
    companydata =[]
    for data in row:
        if col == 5 and notfirstrun:
            companydata.append(data.a.get('href').split('/')[-1])
        else:
            companydata.append(data.text.strip())
        col += 1
    company.append(companydata)
    col =0
    notfirstrun = True

df = pd.DataFrame(company[1:],columns=company[0])
df.rename(columns={'Operations':'Symbol No'},inplace=True)
df.index.name = "SN"
df.drop(columns='',inplace=True)
df.drop(columns='S.N.',inplace=True)
df.to_json('CompanyList.json',orient='index')
print('There are %s Companies'%len(df.index))
df.head()

There are 210 Companies


,Stock Name,Stock Symbol,Sector,Symbol No
SN,,,,
0,Agriculture Development Bank Limited,ADBL,Commercial Banks,397
1,Api Power Company Ltd.,API,Hydro Power,697
2,Arambha Microfinance Bittiya Sanstha Ltd.,AMFI,Microfinance,2777
3,Arun Kabeli Power Ltd.,AKPL,Hydro Power,2757
4,Arun Valley Hydropower Development Co. Ltd.,AHPC,Hydro Power,360


In [4]:
# Getting Company Details
symbol = "NMB"
url = "http://www.nepalstock.com/company/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
table = soup.find("table")
print ("Company: ",table.findAll("td")[0].string)
for row in table.findAll("tr")[4:]:
    col = row.findAll("td")
    print (col[0].string,": ",col[1].string)

Company:  NMB Bank Limited
Address :  Babarmahal
Email Address :  pramod.dahal@nmb.com.np
Website :  http://www.nmb.com.np
Last Traded Price (Rs.) :  312.00
Change (Rs.) and (%) :  4(1.30)
Total Listed Shares :  87,437,842.00
Paid Up Value (Rs.) :  100
Total Paid Up Value (Rs.) :  8,743,784,200.00
Closing Market Price (Rs.) :  312.00
Market Capitalization (Rs.) :  27,280,606,704.00


# Daily FloorSheet Data

In [5]:
DailyFloorSheet="http://www.nepalstock.com.np/main/floorsheet/index/0/?_limit=5000"

http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',DailyFloorSheet)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
FloorSheet=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
for row in rows:
    FloorSheet.append([data.text.strip() for data in row])
FloorSheetdf = pd.DataFrame(FloorSheet[1:-1],columns=FloorSheet[0])
FloorSheetdf.head()

,S.N.,Contract No,Stock Symbol,Buyer Broker,Seller Broker,Quantity,Rate,Amount
0,1,201808286420508,BOKL,28,55,50,288,14400.00
1,2,201808286420509,BOKL,28,6,50,289,14450.00
2,3,201808286420510,SIL,43,25,10,649,6490.00
3,4,201808286420511,LGIL,16,6,10,514,5140.00
4,5,201808286420512,EBL,21,50,31,660,20460.00


In [6]:
df['date'] = pd.to_datetime(FloorSheetdf['Contract No'],format='%Y%m%d%H%M%f').dt.date
df['date']

SN
0      2018-08-28
1      2018-08-28
2      2018-08-28
3      2018-08-28
4      2018-08-28
5      2018-08-28
6      2018-08-28
7      2018-08-28
8      2018-08-28
9      2018-08-28
10     2018-08-28
11     2018-08-28
12     2018-08-28
13     2018-08-28
14     2018-08-28
15     2018-08-28
16     2018-08-28
17     2018-08-28
18     2018-08-28
19     2018-08-28
20     2018-08-28
21     2018-08-28
22     2018-08-28
23     2018-08-28
24     2018-08-28
25     2018-08-28
26     2018-08-28
27     2018-08-28
28     2018-08-28
29     2018-08-28
          ...    
180    2018-08-28
181    2018-08-28
182    2018-08-28
183    2018-08-28
184    2018-08-28
185    2018-08-28
186    2018-08-28
187    2018-08-28
188    2018-08-28
189    2018-08-28
190    2018-08-28
191    2018-08-28
192    2018-08-28
193    2018-08-28
194    2018-08-28
195    2018-08-28
196    2018-08-28
197    2018-08-28
198    2018-08-28
199    2018-08-28
200    2018-08-28
201    2018-08-28
202    2018-08-28
203    2018-08-28
204    

# CompanyStocks Transactions

In [7]:
def CompanyStocksTransactions(SymbolNo,startDate):
    url="http://www.nepalstock.com.np/company/transactions/%s/0/?startDate=%s&endDate=&_limit=9000000"%(SymbolNo,startDate)
    print("Connecting to %s "%url)
    http = urllib3.PoolManager()
    http.addheaders = [('User-agent', 'Mozilla/61.0')]
    web_page = http.request('GET',url)
    print("Adding to DataFrame")
    soup = BS(web_page.data, 'html5lib')
    table = soup.find('table')
    FloorSheet=[]
    rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
    for row in rows:
        FloorSheet.append([data.text.strip() for data in row])
    FloorSheetdf = pd.DataFrame(FloorSheet[1:],columns=FloorSheet[0])
    FloorSheetdf['Date']=pd.to_datetime(dfNepse['Contract No'], format='%Y%m%d%H%M%f', errors='ignore')
    return FloorSheetdf

In [8]:

startDate= '2001-1-1'
count = 0
for symbol in list(df['Symbol No']):
    print("Stock No: %s\nTime: %s"%(symbol,time.ctime()))
    filename = "./data/NEPSE%s.csv"%symbol
    count +=1
    if os.path.isfile(filename):
        print("Stock No: %s Data Already Downloaded"%symbol)
    else:
        dftest=CompanyStocksTransactions(symbol,startDate)
        dftest.to_csv(filename)
    print("%s of %s completed \n"%(count,len(df['Symbol No'])))

Stock No: 397
Time: Tue Aug 28 18:15:38 2018
Connecting to http://www.nepalstock.com.np/company/transactions/397/0/?startDate=2001-1-1&endDate=&_limit=9000000 
Adding to DataFrame


NameError: name 'dfNepse' is not defined

In [ ]:
# plot
# Need to convert Contract Number to Date-time
Stock = 'EBL'
symbol = "%d"%df[df['Stock Symbol']==Stock]['Symbol No']
print(symbol)
dfNepse=pd.read_csv("./data/NEPSE%s.csv"%symbol)
dfNepse.sort_values('Contract No',inplace = True)
startfrom=200101016341249
dfNepse=dfNepse[dfNepse['Contract No']>startfrom]
dfNepse.plot(x='Contract No', y = 'Rate')

In [ ]:
dfNepse.describe()

In [13]:
symbol = "NMB"
url = "http://nepalstock.com/marketdepth/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
tables = soup.findAll("table")
for table in tables:
    for row in table.findAll("tr")[4:]:
        col = row.findAll("td")
        print (col[0].string,": ",col[1].string)

In [1]:
soup.findAll("table")

NameError: name 'soup' is not defined